# Scikic API v2.0
 
## Overview

The scikic api is an inference tool which takes a set of question/answer items and then queries a series of local and remote databases to generate conditional probability distributions over various features. The api is highly modular, and some modules don't use this probabilistic framework, for example the music module simply contacts api.bandsintown.com to provide useful suggestions about local bands to go and see.

The conditional probabilities are combined using a Bayesian network, using the pyMC module. Each module can provide pyMC 'features' which create functions to output the relevant probability distributions.

## Question/Answer dictionaries

The questions and answers are organised to be in 4 value tuples, containing:

- dataset: lets the system know which class to instantiate etc, examples: postcode, census, movielens, ...etc
- dataitem: used by classes to know which aspect of the dataset. For example in the movielens dataset, one could be interested in whether the user's seen a film or what rating they've given the film.
- detail: often unused by the classes, could be, for example, the id of the film we want to know about.
- answer: the user's answer.

## Future Changes/TODO list

Heads-up about a couple of changes:

 - ~~Vasily suggested I move the 'action' selection out and make it part of the URL, to make it more RESTful.~~ DONE.
 - ~~I'll be combining the API query that gets the question string, with the one that generates the 'raw' question tuple. Originally the API was designed when it was run on the same server as the code that was querying it, so extra API calls didn't cost much. As the calls will now be remote, they need combining.~~ DONE
 - ~~document the new API~~ DONE (below)
 - fix security issues (stop outputting debug info to client)
 - add useful error messages to API's output
 - incorporate Cambridge's code
 - write a new install script for the setup
 - install on front end
 - The version number will be used in future.
 - The API doesn't currently check the key. Obviously this will be implemented in future.
 - Get the IP->Location working again (API change?)
 - Check load balancer to backend API still working (this shouldn't be need altering)
 - Set up SSL for API load balancer (previously left connection between frontend and backend non-SSL as it's all in the AWS datacentre).

## Available actions

Here are some examples of the API in action. A **POST** request is used for the query, in case the data we're sending is too large to fit in a GET request. The POST body is json.

### 1. question

#### Parameters
One passes to this call in data, a dictionary containing:
 - 'questions_asked'
 - 'unprocessed_questions'
 - 'facts'
 - 'target'

The 'questions_asked' include all the questions we've asked, so we don't ask the same question again.
The 'facts' dictionary contains information that we've found from earlier questions, etc. It allows caching of the calculations from earlier calls to the API.
The 'unprocessed_questions' are a list of question/answers that we've asked before, that haven't had their results added to the 'facts' dictionary.
The 'target' item is currently unused, but in the future will allow the choice of question to be selected to maximise the information about a particular feature.

#### Returns

This call returns a dictionary containing three things:

 - a 'facts' dictionary - this you can pass back in future so that the method doesn't have to recalculate or generate earlier results.
 - a 'question' dictionary, containing the dictionary describing the question, e.g. {dataset,dataitem,detail}.
 - a 'question_string' dictionary, containing:
 
    - 'text' - the actual string of the question (e.g. "Who's your favourite band or artist?")
    - 'type' - the type of question (it might just want a text reply, so this would equal 'text' or it might be a choice, and so would say 'select'
    - 'options' - optional, and is included if the type is 'select'.


#### Usage example

1. One might initially call this method with all these fields being empty. The method will return an empty 'facts' dictionary and a question dictionary for the first question you want to ask. 
2. Once you have an answer from the user you would call the method again, this time with the question/answer tuple as both 'questions_asked' and 'unprocessed_questions'.
3. The method will return a facts dictionary now, potentially with some results from the processing of the last answer, and another question for you to ask. 
4. When you call the method a third time (with the user's second answer), you'll pass all the question/answer tuples that you've asked so far in 'questions_asked' and the last question/answer tuple in 'unprocessed_questions'. You'll also pass the new facts dictionary, that now has some content in it.
5. This process continues, with the facts dictionary growing each time, the 'questions_asked' growing too, and each time you just have one item in 'unprocessed_questions'.

To summarise:

Generating a question requires a dictionary of 'questions_asked', 'facts' and 'target'. The 'questions_asked' is a list of dictionaries of previous questions, that you want to avoid asking again.
The 'unprocessed_questions' are questions that you've asked already and that haven't been incorporated into the 'facts' dictionary.

Below is an example:

In [5]:
#The url of the API currently (in the long run we should give the backend a domain, maybe api.scikic.org?)
#apiurl = 'http://api.scikic.org' #production api
apiurl = 'http://dev.scikic.org' #development api

In [19]:
import requests
#We provide data about previous questions etc:
#data consists of a dictionary of (all of these are optional):

#'questions_asked': An array of previous questions and answers we've asked, consists of a list of dictionaries.
questions_asked = [{'dataset':'postal','dataitem':'postcode','detail':''},{"detail": "", "dataitem": "favourite_artist", "dataset": "music"}]

#none of the questions that have been asked before have been processed.
unprocessed_questions = questions_asked

#'facts': If you've run the inference query and stored a copy of the facts dictionary you can pass it back.
#this makes it quicker. In this case we've not yet had any questions processed.
facts = {}

#'target': What feature we want to know more about (example: 'age', 'gender', 'location') NOT YET IMPLEMENTED
#not used. All these items are optional, so we just don't include it.

#Build the dictionary:
data = {'unprocessed_questions':unprocessed_questions,'questions_asked':questions_asked,'facts':facts}

#put it into the payload of the request. This also includes the version, the api key we're using and the action we want (in this case generate a question)
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE', 'action':'question'}
r = requests.post(apiurl+'/question',json=payload)
r.content

'{"facts": {"guess_loc": {}, "where": {}}, "question_string": {"type": "select", "question": "Which country are you in now?", "options": ["United States", "United Kingdom", "Germany", "Canada", "France", "Other"]}, "question": {"detail": "{}", "dataitem": "country", "dataset": "geoloc"}}'

Here it has output:

 - the facts dictionary: Empty
 - the question it suggests we ask, which in an earlier case is from the movielens dataset class, and is asking if they've seen movie number 2541.
 - the question string, with the type of question and options (if relevant). For example the dictionary `{'dataset':'postal','dataitem':'postcode','detail':''}` gets converted to the question "`What's your postcode?`" with type "`text`" (i.e. the user can type anything).

### 2. inference

#### Parameters

The data dictionary should contain three things, similar to the "action: question" above,

- questions_asked - list of question tuples that we've asked (with their answers)
- unprocessed_questions - list of question tuples that we've asked (with their answers), that have not yet been added to the facts dictionary.
- facts - the current 'facts' dictionary (possibly provided by earlier calls using action:question)

Returns a dictionary of:

##### features
This is a dictionary of things that have probabilities associated, for example one of its items is 'household' with the following fields:
 
     {"distribution": [0.029, 0.058, 0.23, 0.034, 0.070, 0.026, 0.055, 0.036, 0.14, 0.024, 0.023, 0.035, 0.24], "quartiles": {"upper": 11, "lower": 2, "mean": 6.46}}
 
 where the distribution is how likely the person is to be in each of the categories of a household (these categories can be found in the metadata from the module, or elsewhere). The quartiles don't mean much here as this is properly categorical data. This makes more sense in data such as age.
 
##### facts
As mentioned previously is a set of truths about the user generated from processing their answers.

##### Insights
This is a list of strings, generated by each module, here is an example:
 
     ["I can\'t tell which country you\'re in, just looking at your facebook likes, as I can\'t see your facebook likes!", "You are aged between 20 and 33.", "You don\'t have children living at home", " I think you are Christian or of no religion."]}
 
Note regarding the distribution above: If some probabilities are zero towards the end of a list then the list will be truncated. For example if inference is certain the user is a male, then the output list will be {"factor_gender":[1.0]}. If they are definitely female it will be {"factor_gender":[0.0, 1.0]}

#### Example call using action:inference

Below we set up the data dictionary with one question asked (and answered) regarding postcode. It also contains the same question/answer dictionary as an unprocessed question. The facts dictionary is empty.

The output

In [21]:
import requests
import json
postcode = 's63af'
questions_asked = []

#their address:
questions_asked.append({'dataset':'postal','dataitem':'postcode','detail':'','answer':'s63af'})

#Additional info from facebook:
fb_data = json.dumps({'reply':json.dumps({'first_name':'Alan'})})
questions_asked.append({'dataset':'facebook','dataitem':'','detail':'','answer':fb_data})

#we assume that we've not processed any of these questions yet. 
#You'll only not do this if you've asked some of these already,
#and populated the 'facts' dictionary.
unprocessed_questions = questions_asked[:]
facts = {}
data = {'questions_asked':questions_asked,'unprocessed_questions':unprocessed_questions,'facts':facts}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/inference',json=payload)
#print r.content
result = json.loads(r.content)
print "==Facts=="
print result['facts']
print ""
print "==Features=="
for feat in result['features']:
    print feat
    print result['features'][feat]
print ""
print "==Text insights=="
print result['insights']

==Facts==
{u'guess_loc': {}, u'first_name': u'Alan', u'where': {u'ukcensus': [{u'item': u'E00172420', u'probability': 1.0, u'level': u'oa'}], u'latlong': [{u'item': [53.389942, -1.476789], u'probability': 1.0}], u'country': [{u'item': u'gb', u'probability': 1.0}], u'city': [{u'item': [u'Sheffield', u'uk'], u'probability': 1.0}]}, u'where_history': {u'error': u'no_fb_likes'}}

==Features==
religion
{u'distribution': [0.24977777777777777, 0.03955555555555555, 0.043111111111111114, 0.035555555555555556, 0.04844444444444444, 0.04488888888888889, 0.03688888888888889, 0.5017777777777778], u'quartiles': {u'upper': 7, u'lower': 1, u'mean': 4.384444444444444}}
household
{u'distribution': [0.03111111111111111, 0.028888888888888888, 0.19777777777777777, 0.033777777777777775, 0.027555555555555555, 0.03777777777777778, 0.043555555555555556, 0.049777777777777775, 0.29333333333333333, 0.032, 0.024888888888888887, 0.04133333333333333, 0.1582222222222222], u'quartiles': {u'upper': 9, u'lower': 2, u'mea

For the example above one can see that after just processing the answer about postcode, quite a bit of new info has been generated.

First the facts dictionary. This is often dataset specific stuff, although I've tried to make things compatable between datasets.

 - 'guess_loc': {} - info on whether it guessed the location of the user from IP address (I think)
 - 'where': - the dictionary describing the location of the user's home. This is quite tricky, as different sources of data have different resolutions, etc about this.
     - {'ukcensus': [{'item': 'E00172420', 'probability': 1.0, 'level': 'oa'}] - In terms of the UK census, with have a list of output areas for this person's home. This list only has one item in it. Each item in the list has a probability associated, in this case the probability is 1.0: We are certain the person is in that output area.
     - 'country': [{'item': 'gb', 'probability': 1.0}] - Which country they're in. A list of countries with associated probabilities.
     - 'city': [{'item': ['Sheffield', 'uk'], 'probability': 1.0}]} - which city their in (with probabilites).
     - 'where_history': {u'error': u'no_fb_likes'}} - if we have access to facebook likes, it tries to generate a history of where the person's lived. But the error item means that it's not managed to get hold of the likes to do this.

Next the features dictionary. Different datasets provide different conditional probability distributions. Each distribution has at least two features associated. If they are not in the list already the module adds them, thus one has a list of features at the end.

The value of these features is then estimated using MCMC with the pyMC module.

The example above has five features: religion, household, (census) output area, gender and age. They're all catagorical (for now) although that's purely due to the type of data that we have about them.

Looking just at the household feature: [0.022222222222222223, 0.05688888888888889, 0.23466666666666666, 0.03244444444444444, 0.08266666666666667, 0.028, 0.059111111111111114, 0.03955555555555555, 0.14222222222222222, 0.024, 0.024444444444444446, 0.028444444444444446, 0.22533333333333333]

Each value refers to the probability of being of a given type of household (the labels associated are available via the metadata of the module).

Finally there are insights, these are simple strings of facts about the person:
 - "I can't tell which country you're in, just looking at your facebook likes, as I can't see your facebook likes!" - this first one is more of an error message warning us that we couldn't get to their facebook like data.
 - "You are aged between 19 and 31"
 - "I think you are Christian or of no religion"
 
Here's another example, with an american zip code:

In [15]:
import requests
questions_asked = [{'dataset':'postal','dataitem':'zipcode','detail':'','answer':'86021'}]
unprocessed_questions = [{'dataset':'postal','dataitem':'zipcode','detail':'','answer':'86021'}]
facts = {}
        
data = {'questions_asked':questions_asked,'unprocessed_questions':unprocessed_questions,'facts':facts}

payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/inference',json=payload)
r.content

'{"facts": {"guess_loc": {}, "where": {}, "where_history": {"error": "no_fb_likes"}}, "features": {"factor_gender": {"distribution": [0.3337777777777778, 0.6662222222222223], "quartiles": {"upper": 1, "lower": 0, "mean": 0.6662222222222223}}, "factor_age": {"distribution": [0.003111111111111111, 0.0071111111111111115, 0.012, 0.008444444444444444, 0.012888888888888889, 0.008444444444444444, 0.0057777777777777775, 0.014666666666666666, 0.008888888888888889, 0.014666666666666666, 0.010222222222222223, 0.015111111111111112, 0.010222222222222223, 0.009777777777777778, 0.008, 0.008888888888888889, 0.013777777777777778, 0.010666666666666666, 0.010666666666666666, 0.010666666666666666, 0.011555555555555555, 0.008888888888888889, 0.012888888888888889, 0.014666666666666666, 0.011111111111111112, 0.010666666666666666, 0.008444444444444444, 0.011555555555555555, 0.008, 0.010666666666666666, 0.013777777777777778, 0.008444444444444444, 0.011555555555555555, 0.008888888888888889, 0.007555555555555556

You can see this is similar, except the 'where' item in the dictionary has a 'uscensus' item within it. This contains two items:

    {"item": ["04", "015", "950100", "1"], "probability": 0.514, "level": "blockgroup"}
    {"item": ["04", "015", "950100", "3"], "probability": 0.486, "level": "blockgroup"}

Because zipcodes cover quite large areas, it doesn't know which blockgroup the person's home is in, as the zip code spans more than one blockgroup. It therefore gives the probability of being in the two.

The US census module doesn't know about religion, so doesn't have a conditional probability distribution about it, so no religion feature is created. The features that are created are:

 - bg (block group)
 - gender
 - age

### 3. Metadata *[action: metadata]*

Some of the classes provide metadata about the results. Use the 'metadata' action to retrieve these. Pass a dictionary in 'data' with the name of the dataset, or leave empty to get all the metadata of all the classes.

In this example we display the citation information for the 'babynames' dataset.

In [22]:
import requests
import json
data = {'dataset':'babynames'}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/metadata',json=payload)
for item in json.loads(r.content):
    if 'citation' in item:
        print(item['citation'])

The ONS provide statistics on the distribution of the names of baby's in the UK: <a href="http://www.ons.gov.uk/ons/about-ons/business-transparency/freedom-of-information/what-can-i-request/published-ad-hoc-data/pop/august-2014/baby-names-1996-2013.xls">1996-2013</a> and <a href="http://www.ons.gov.uk/ons/rel/vsob1/baby-names--england-and-wales/1904-1994/top-100-baby-names-historical-data.xls">1904-1994</a>.


In this example we get all citations:

In [23]:
import requests
import json
data = {}#no dataset specified (makes it output all metadata)
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/metadata',json=payload)
for item in json.loads(r.content):
    if 'citation' in item:
        print(item['citation'])

The <a href="facebook.com">facebook</a> graph API
The <a href="http://www.census.gov/developers/">US census bureau</a>. In particular the American Community Survey <a href="http://www.census.gov/data/developers/data-sets/acs-survey-5-year-data.html">5 year data</a>.
The <a href="http://files.grouplens.org/datasets/movielens">movielens</a> database
The ONS provide statistics on the distribution of the names of baby's in the UK: <a href="http://www.ons.gov.uk/ons/about-ons/business-transparency/freedom-of-information/what-can-i-request/published-ad-hoc-data/pop/august-2014/baby-names-1996-2013.xls">1996-2013</a> and <a href="http://www.ons.gov.uk/ons/rel/vsob1/baby-names--england-and-wales/1904-1994/top-100-baby-names-historical-data.xls">1904-1994</a>.
The <a href="https://geoportal.statistics.gov.uk">UK office of national statistics</a> (see <a href="http://www.ons.gov.uk/ons/guide-method/geography/products/census/lookup/other/index.html">details</a> and <a href="https://geoportal.stat

## Typical API usage

The scikic front end may use the API in the following way.

In [29]:
import requests

#We start with no questions asked, none unprocessed, and nothing in the facts dictionary.
questions_asked = []
unprocessed_questions = []
facts = {}

for loop in range(3): #we'll ask three questions
    
    #1. get the question (populate the data dictionary & send it off)
    data = {'unprocessed_questions':unprocessed_questions,'questions_asked':questions_asked,'facts':facts}
    payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
    r = requests.post(apiurl+'/question',json=payload) #>>>
    question_query_result = json.loads(r.content)
    
    #if processing was done then more items will be available for 'facts':
    facts = question_query_result['facts']
    question = question_query_result['question']
    #ask the user this question    
    userinput = raw_input(question_query_result['question_string']['question'])
    question['answer'] = userinput #add their answer

    #add this to the list of questions we've asked, and unprocessed questions
    questions_asked.append(question)
    unprocessed_questions.append(question)
    
#3. Once enough questions are asked we can do inference.
#   Populate the data dictionary with questions asked,
#   unprocessed questions and facts (as for the question query in step 1)
data = {'questions_asked':questions_asked,'unprocessed_questions':unprocessed_questions,'facts':facts}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE'}
r = requests.post(apiurl+'/inference',json=payload)
inference_results = json.loads(r.content)

#It generates insights from these questions, which are displayed below.
print "\nInsights\n"
for insight in inference_results['insights']:
    print insight

What's your favourite band or artist? (be honest!)eels
Have you seen Insider, The (1999)? (yes or no)no
Have you seen Mad Max Beyond Thunderdome (1985)? (yes or no)no

Insights

Note: I can't see your facebook likes.
OSM Answer system online


# Scikic System

## Instances

I'm not currently working on the front end, as that's being taken over by Vasily. The latest API server is running on the development server: dev.scikic.org.

An autoscaling group, launch configuration and load balancer are used to provide the backend processing on the production machines. This will include SSL. The launch configuration uses the 'Production BackEnd Image' instance to produce the AMI.

### Recipe for self to update backend

1. On the dev system run nosetest.
2. Push the changes to git
3. On the production backend image server, pull the changes.
4. Right click on server, create image.
5. image name: production-backend-120116.

6. Click on launch configurations
7. Create Launch Cofiguration
8. Select the new image name you just created
9. Select t2.medium
10. Name: production-backend-120116-lc
11. Next> Delete both on termination.
12. Select existing security group (production backend)
13. Launch

14. Autoscaling group. Select the production-backend-4-asg and click edit
15. Select the new launch configuration


## DNS

DNS is provided by namecheap (see Neil for username/password). I've set the nameservers for scikic.org to point to the AWS Route53 DNS system.

## How to configure in the future

- cloudinit


# Stress Testing - Out of Date

To test how well the system copes under load, this script calls the API multiple times:

In [30]:
import requests
import time
from threading import Thread

def myfunc(i):
    print "(launch)"
    questions_asked = [{'dataset':'postal','dataitem':'postcode','detail':'','answer':'s63af'}]
    unprocessed_questions = [{'dataset':'postal','dataitem':'postcode','detail':'','answer':'s63af'}]
    facts = {}
        
    data = {'questions_asked':questions_asked,'unprocessed_questions':unprocessed_questions,'facts':facts}
    payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE', 'action':'inference'}
    url = "http://api.scikic.org"
    r = requests.post(url,json=payload)
    print r.content

it = 240
for itn in range(1,600):
    print "Launching %d processes over the next 120 seconds" % it
    for i in range(it):
        t = Thread(target=myfunc, args=(i,))
        t.start()
        time.sleep(120./it)


Launching 240 processes over the next 120 seconds
(launch)
(launch)
(launch)
(launch)
(launch)


KeyboardInterrupt: 

## Questions

- What happens if a class doesn't return a None question (but instead actually has a question)? But it also needs processing?